In [ ]:
# export
import json,fire,re,os,shutil,glob
from pathlib import Path

In [ ]:
# default_exp export

## Find our way

When exporting things the defualt behavior is that a cell is exported to a module of the same name as the file. However our notebook might be in the base directory (core module) or in a subfolder (application for instance) and we need the export command to reproduce that architecture of directories.

The root directory is cleary identified by having a .root file in it.

In [ ]:
# export
def is_root(path): return (Path(path)/'.root').exists()

`path` can be a string or a pathlib oject.

In [ ]:
# test
assert is_root('.')
assert is_root(Path('.'))
assert not is_root(Path.home())

Then we need to know, for any file, the structure relative to root to find the destination to export it.

In [ ]:
# export
def export_dest(fname):
    "Returns the destination for an export coming from `fname`"
    fn = Path(fname)
    while not is_root(fn): fn = fn.parent
    return fn/'fastai'/Path(fname).relative_to(fn)

`fname` can be a string or Pathlib object, then the function scans for where the root directory is to return a path similar to `fname`, with fastai in the middle.

In [ ]:
# test
os.makedirs('test', exist_ok=True)
assert export_dest('test') == Path('fastai/test')
assert export_dest(Path('test')) == Path('fastai/test')
assert export_dest('test/module.py') == Path('fastai/test/module.py')
assert export_dest('test/deep/module.py') == Path('fastai/test/deep/module.py')
shutil.rmtree('test')

## Reading a notebook

### Finding export cells

A jupyter notebook is a json file behind the scenes. We can just read it with the json module, which will return a nested dictionary of dictionaries/lists of dictionaries.

In [ ]:
test_nb = json.load(open('00_export.ipynb'))

The root has four keys: `cells` contains the cells of the notebook, `metadata` some stuff around the version of python used to execute the notebook, `nbformat` and `nbformat_minor` the version of nbformat. 

In [ ]:
test_nb.keys()

dict_keys(['cells', 'metadata', 'nbformat', 'nbformat_minor'])

In [ ]:
test_nb['metadata']

{'kernelspec': {'display_name': 'Python 3',
  'language': 'python',
  'name': 'python3'},
 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
  'file_extension': '.py',
  'mimetype': 'text/x-python',
  'name': 'python',
  'nbconvert_exporter': 'python',
  'pygments_lexer': 'ipython3',
  'version': '3.7.1'}}

In [ ]:
f"{test_nb['nbformat']}.{test_nb['nbformat_minor']}"

'4.2'

The cells key then contains a list of cells. Each one is a new dictionary that contains entries like the type (code or markdown), the source (what is written in the cell) and the output (for code cells).

In [ ]:
test_nb['cells'][0]

{'cell_type': 'code',
 'execution_count': 1,
 'metadata': {},
 'outputs': [],
 'source': ['# export\n',
  'import json,fire,re,os,shutil\n',
  'from pathlib import Path']}

This function will be used to detect different `# code` at the beginning of cells.

In [ ]:
# export
def check_re_pattern(cell, pat):
    "Check if `cell` contains given `pat`."
    if cell['cell_type'] != 'code': return False
    src = cell['source']
    if len(src) == 0: return False
    return re.match(pat, src[0], re.IGNORECASE) 

Now we can check for the cell that need to be exported.

In [ ]:
# export
def is_export(cell, default):
    "Check if `cell` is to be exported and returns the name of the module."
    if check_re_pattern(cell, r'^\s*#\s*exports?\s*$'): return default
    tst = check_re_pattern(cell, r'^\s*#\s*exports?\s*(\S+)\s*$')
    return os.path.sep.join(tst.groups()[0].split('.')) if tst else None 

We check if the cell begins with an `#export` or `#exports` code, potentially with a module name where we want it exported. The default is given in a cell of the form `#default_exp bla` inside the notebook (usually at the top), though in this function, it needs the be passed (the final script will read the whole notebook to find it).

In [ ]:
cell = test_nb['cells'][0].copy()
assert is_export(cell, 'export') == 'export'
cell['source'][0] = "# exports" 
assert is_export(cell, 'export') == 'export'
cell['source'][0] = "# export mod" 
assert is_export(cell, 'export') == 'mod'
cell['source'][0] = "# export mod.file" 
assert is_export(cell, 'export') == 'mod/file'
cell['source'][0] = "# expt mod.file"
assert is_export(cell, 'export') is None
cell['source'][0] = "# export"

### Finding the default export path

In [ ]:
def find_default_export(cells):
    for cell in cells:
        tst = check_re_pattern(cell, r'^\s*#\s*default_exp\s*(\S*)\s*$')
        if tst is not None: return tst.groups()[0]

In [ ]:
find_default_export(test_nb['cells'])

'export'

In [ ]:
def getSortedFiles(allFiles, upTo=None):
    ""
    if (allFiles==True): ret = glob.glob('*.ipynb') # Checks both that is bool type and that is True
    else: ret = glob.glob(allFiles) if isinstance(allFiles,str) else []
    if len(ret)==0: 
        print('WARNING: No files found')
        return ret
    if upTo is not None: ret = [f for f in ret if str(f)<=str(upTo)]
    return sorted(ret)

In [ ]:
def notebook2script(fname=None, allFiles=None, upTo=None):
    # initial checks
    if (allFiles is None) and (upTo is not None): allFiles=True # Enable allFiles if upTo is present
    if (fname is None) and (not allFiles): print('Should provide a file name')
    if not allFiles: notebook2scriptSingle(fname)
    else:
        print('Begin...')
        [notebook2scriptSingle(f) for f in getSortedFiles(allFiles,upTo)]
        print('...End')

   '''Finds cells starting with `#export` and puts them into a new module
       + allFiles: convert all files in the folder
       + upTo: convert files up to specified one included
       
       ES: 
       notebook2script --allFiles=True   # Parse all files
       notebook2script --allFiles=nb*   # Parse all files starting with nb*
       notebook2script --upTo=10   # Parse all files with (name<='10')
       notebook2script --allFiles=*_*.ipynb --upTo=10   # Parse all files with an '_' and (name<='10')

In [ ]:
#export 
def notebook2scriptSingle(fname):
    "Finds cells starting with `#export` and puts them into a new module"
    fname = Path(fname)
    nb = json.load(open(fname,'r'))
    default = find_default_export(nb['cells'])
    default = os.path.sep.join(default.split('.'))
    fname_out = Path.cwd()/'fastai'/f'{default}.py'
    code_cells = [c for c in nb['cells'] if is_export(c, default) is not None]
    module = f'''
#################################################
### THIS FILE WAS AUTOGENERATED! DO NOT EDIT! ###
#################################################
# file to edit: dev/{fname.name}

'''
    for cell in code_cells: module += ''.join(cell['source'][1:]) + '\n\n'
    # remove trailing spaces
    module = re.sub(r' +$', '', module, flags=re.MULTILINE)
    output_path = fname.parent/'exp'/fname_out
    open(output_path,'w').write(module[:-2])
    print(f"Converted {fname} to {output_path}")

In [ ]:
notebook2scriptSingle('00_export.ipynb')

Converted 00_export.ipynb to /home/ubuntu/fastai_docs/dev/fastai/export.py


In [ ]:
if __name__ == '__main__': fire.Fire(notebook2script)